In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

genai.configure(api_key=os.environ.get('GEMINI_API_KEY'))

## Model

We build a class to handle API calls to the LLM, in case we decide to change the LLM model.

Please make sure you have obtained the appropriate API key before continuing.

In [3]:
class AbstractModel:
    def __init__(self, base: str):
        pass

    def query(self, prompt: str) -> str:
        raise NotImplementedError

class QueryException(Exception):
    def __init__(self, message: str):
        super().__init__(f"Failed to query to the LLM: {message}")

In [4]:
class GeminiModel(AbstractModel):
    def __init__(self, base: str):
        self.base = base
        self.client = genai.GenerativeModel('gemini-1.5-pro')

    def query(self, prompt: str) -> str:
        response = self.client.generate_content(self.base.format(prompt=prompt)).candidates[0]
        if not hasattr(response, 'content'):
            raise QueryException('LLM response does not have content')
        return response.content.parts[0].text

## Prompt

We design the prompts and attempt to get the answer here.

### Category

The following prompt is designed to get the suitable category of product.

In [5]:
CATEGORY_PROMPT = """
<instruction>
You are a sales assistant on a large e-commerce platform.
Your job is to recommend the correct product to the customers.

The customer is finding a suitable gift.
The specific description of what the user wants is given in the "userquery" tag below.
You are to determine an appropriate category of product that the user can buy as gift.

The category you select should be listed in the "categories" tag below.
You must start your response with one of the categories below.
You are not to format the category in any way.
Failure to start your response with one of the categories below will lead to catastrophic effects.

You are then to explain your choice of category on the next line.
</instruction>

<categories>
Hobbies & Books
Books
Music, Movies & Games
Music Instruments
Figures & Model Kits
Electronics
Cameras
Fashion
Toys
Collectibles
</categories>

<userquery>
{prompt}
</userquery>
"""

In [9]:
gemini_category_model = GeminiModel(CATEGORY_PROMPT)

In [10]:
gemini_category_model.query("A gift for my friend, who loves to read Harry Potter.")

"Books \nThis is the most fitting category as it directly relates to your friend's love for Harry Potter, which is a book series. \n"

In [12]:
gemini_category_model.query("How are you today?")

"Hobbies & Books \nThis is too broad of a request to recommend a specific category. I need more information! What are this person's interests? What do they like to do for fun?  Tell me more and I can give you a more tailored suggestion! 😊 \n"